### Making Predictions, Preparing a Kaggle Submission File

Submissions should follow the kaggle given format, which is two columns (comma-separated).
The first column is the air_store_id, which is concatenated with the visit date. The second is the prediction of number of visitors to the specific restaurant.

The prediction dates are: 2017-04-23 through 2017-05-31.

Each air_store_id should have info. for each date.

In [1]:
#setup
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import missingno

aReserveDF = pd.read_csv('air_reserve.csv', parse_dates = ['visit_datetime', 'reserve_datetime']) 
aVisitDF = pd.read_csv('air_visit_data.csv', parse_dates = ['visit_date']) 
aStoreDF = pd.read_csv('air_store_info.csv')

hReserveDF = pd.read_csv('hpg_reserve.csv', parse_dates = ['visit_datetime', 'reserve_datetime']) 
hStoreDF = pd.read_csv('hpg_store_info.csv') 

dateInfoDF = pd.read_csv('date_info.csv', parse_dates = ['calendar_date'])

sampleSubmissionDF = pd.read_csv('sample_submission.csv') 

storeIdRelationDF = pd.read_csv('store_id_relation.csv') 
hReserveDF['visit_year'] = hReserveDF['visit_datetime'].dt.year
hReserveDF['visit_month'] = hReserveDF['visit_datetime'].dt.month
hReserveDF['visit_day'] = hReserveDF['visit_datetime'].dt.day
hReserveDF['reserve_year'] = hReserveDF['reserve_datetime'].dt.year#
hReserveDF['reserve_month'] = hReserveDF['reserve_datetime'].dt.month
hReserveDF['reserve_day'] = hReserveDF['reserve_datetime'].dt.day
#hReserveDF.drop(['visit_datetime','reserve_datetime'], axis=1, inplace=True)

hReserveDF = hReserveDF.groupby(['hpg_store_id', 'visit_year', 'visit_month','visit_day','reserve_year','reserve_month','reserve_day', 'reserve_datetime', 'visit_datetime'], as_index=False).sum()
aReserveDF['visit_year'] = aReserveDF['visit_datetime'].dt.year
aReserveDF['visit_month'] = aReserveDF['visit_datetime'].dt.month
aReserveDF['visit_day'] = aReserveDF['visit_datetime'].dt.day
aReserveDF['reserve_year'] = aReserveDF['reserve_datetime'].dt.year
aReserveDF['reserve_month'] = aReserveDF['reserve_datetime'].dt.month
aReserveDF['reserve_day'] = aReserveDF['reserve_datetime'].dt.day

#aReserveDF.drop(['visit_datetime','reserve_datetime'], axis=1, inplace=True)
dateInfoDF['calendar_year'] = dateInfoDF['calendar_date'].dt.year
dateInfoDF['calendar_month'] = dateInfoDF['calendar_date'].dt.month
dateInfoDF['calendar_day'] = dateInfoDF['calendar_date'].dt.day
#dateInfoDF.drop(['calendar_date'], axis=1, inplace=True)
aVisitDF['visit_year'] = aVisitDF['visit_date'].dt.year
aVisitDF['visit_month'] = aVisitDF['visit_date'].dt.month
aVisitDF['visit_day'] = aVisitDF['visit_date'].dt.day
aVisitDF.drop(['visit_date'], axis=1, inplace=True)

hReserveDF = pd.merge(hReserveDF, storeIdRelationDF, on='hpg_store_id', how='inner')
hReserveDF.drop(['hpg_store_id'], axis=1, inplace=True)
aReserveDF = pd.concat([aReserveDF, hReserveDF])
aReserveDF = aReserveDF.groupby(['air_store_id', 'visit_year', 'visit_month','visit_day', 'visit_datetime', 'reserve_datetime'],\
                         as_index=False).sum().drop(['reserve_day','reserve_month','reserve_year'], axis=1)
aReserveDF = pd.merge(aReserveDF, dateInfoDF, left_on=['visit_year','visit_month','visit_day'], right_on=['calendar_year','calendar_month','calendar_day'], how='left')
aReserveDF.drop(['calendar_year','calendar_month','calendar_day'], axis=1, inplace=True)
aReserveDF = pd.merge(aReserveDF, aStoreDF, on='air_store_id', how='left')
trainDF = pd.merge(aReserveDF, aVisitDF, on=['air_store_id','visit_year','visit_month','visit_day'], how='left')
trainDF.fillna(0,inplace=True)

trainDF.sort_values(by=['visit_year','visit_month', 'visit_day', 'air_store_id'],ascending=[True,True,True,True],inplace=True)
grouped=trainDF.groupby(['visit_year','visit_month', 'visit_day','air_store_id','visitors', 'day_of_week', 'holiday_flg', 'air_genre_name'], as_index=False)['reserve_visitors'].sum()

In [2]:
grouped.head()

,visit_year,visit_month,visit_day,air_store_id,visitors,day_of_week,holiday_flg,air_genre_name,reserve_visitors
0,2016,1,1,air_08cb3c4ee6cd6a22,0.0,Friday,1,Izakaya,2
1,2016,1,1,air_877f79706adbfb06,3.0,Friday,1,Japanese food,3
2,2016,1,1,air_db4b38ebe7a7ceff,21.0,Friday,1,Dining bar,9
3,2016,1,1,air_db80363d35f10926,8.0,Friday,1,Dining bar,9
4,2016,1,2,air_08cb3c4ee6cd6a22,0.0,Saturday,1,Izakaya,6


### Create submission file

The submission file should be in the format as seen in the sampleSubmissionDF.head() output below. The id is a concatenation of the air_store_id and the calendar date. 

Our current trainDF does not have such concatenation, so we have to undo the concatenation, and later redo it after making a prediction for visitors.

The sample submission file has '0' entered for visitors. This column has to be dropped. Then, a new column will be added with our predictions of number of visitors.

In [3]:
sampleSubmissionDF.head()

,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,0
1,air_00a91d42b08b08d9_2017-04-24,0
2,air_00a91d42b08b08d9_2017-04-25,0
3,air_00a91d42b08b08d9_2017-04-26,0
4,air_00a91d42b08b08d9_2017-04-27,0


In [4]:
sampleSubmissionDF['air_store_id'] = sampleSubmissionDF.id.map(lambda x:'_'.join(x.split('_')[:-1]))

In [5]:
sampleSubmissionDF['calendar_date'] = sampleSubmissionDF.id.map(lambda x:x.split('_')[2])

In [6]:
sampleSubmissionDF.drop('visitors', axis=1,inplace=True)

In [7]:
sampleSubmissionDF.head()

,id,air_store_id,calendar_date
0,air_00a91d42b08b08d9_2017-04-23,air_00a91d42b08b08d9,2017-04-23
1,air_00a91d42b08b08d9_2017-04-24,air_00a91d42b08b08d9,2017-04-24
2,air_00a91d42b08b08d9_2017-04-25,air_00a91d42b08b08d9,2017-04-25
3,air_00a91d42b08b08d9_2017-04-26,air_00a91d42b08b08d9,2017-04-26
4,air_00a91d42b08b08d9_2017-04-27,air_00a91d42b08b08d9,2017-04-27


In [8]:
sampleSubmissionDF['calendar_date'] = pd.DatetimeIndex(sampleSubmissionDF['calendar_date'])


In [9]:
sampleSubmissionDF.head()

,id,air_store_id,calendar_date
0,air_00a91d42b08b08d9_2017-04-23,air_00a91d42b08b08d9,2017-04-23
1,air_00a91d42b08b08d9_2017-04-24,air_00a91d42b08b08d9,2017-04-24
2,air_00a91d42b08b08d9_2017-04-25,air_00a91d42b08b08d9,2017-04-25
3,air_00a91d42b08b08d9_2017-04-26,air_00a91d42b08b08d9,2017-04-26
4,air_00a91d42b08b08d9_2017-04-27,air_00a91d42b08b08d9,2017-04-27


In [11]:
sampleSubmissionDF = sampleSubmissionDF.merge(dateInfoDF,on ='calendar_date', how='left')

In [12]:
sampleSubmissionDF.head()

,id,air_store_id,calendar_date,day_of_week,holiday_flg,calendar_year,calendar_month,calendar_day
0,air_00a91d42b08b08d9_2017-04-23,air_00a91d42b08b08d9,2017-04-23,Sunday,0,2017,4,23
1,air_00a91d42b08b08d9_2017-04-24,air_00a91d42b08b08d9,2017-04-24,Monday,0,2017,4,24
2,air_00a91d42b08b08d9_2017-04-25,air_00a91d42b08b08d9,2017-04-25,Tuesday,0,2017,4,25
3,air_00a91d42b08b08d9_2017-04-26,air_00a91d42b08b08d9,2017-04-26,Wednesday,0,2017,4,26
4,air_00a91d42b08b08d9_2017-04-27,air_00a91d42b08b08d9,2017-04-27,Thursday,0,2017,4,27


In [13]:
sampleSubmissionDF.tail()

,id,air_store_id,calendar_date,day_of_week,holiday_flg,calendar_year,calendar_month,calendar_day
32014,air_fff68b929994bfbd_2017-05-27,air_fff68b929994bfbd,2017-05-27,Saturday,0,2017,5,27
32015,air_fff68b929994bfbd_2017-05-28,air_fff68b929994bfbd,2017-05-28,Sunday,0,2017,5,28
32016,air_fff68b929994bfbd_2017-05-29,air_fff68b929994bfbd,2017-05-29,Monday,0,2017,5,29
32017,air_fff68b929994bfbd_2017-05-30,air_fff68b929994bfbd,2017-05-30,Tuesday,0,2017,5,30
32018,air_fff68b929994bfbd_2017-05-31,air_fff68b929994bfbd,2017-05-31,Wednesday,0,2017,5,31
